In [35]:
#Importing required libraries 
import os
import pandas as pd 
import numpy as np

In [36]:
def compute_annualized_volatility(df):
    """
    Compute annualized volatility from Adj Close prices.
    Uses daily log returns, annualized using sqrt(252).
    """
    df = df.sort_values("Date")
    df["logret"] = np.log(df["Adj Close"] / df["Adj Close"].shift(1))
    
    # Drop the first NA
    df = df.dropna(subset=["logret"])
    
    daily_vol = df["logret"].std()
    annualized_vol = daily_vol * np.sqrt(252)
    return annualized_vol

In [37]:
DATA_FOLDER = "nse_prices"

# Defining Financial Year Windows 
START_DATE = "2020-04-01"
END_DATE   = "2025-03-31"

# Creating a List of tickers (same as imported filenames without .csv)
TICKERS = [
    "SBIBANK",
    "BANKBARODA",
    "CANBK",
    "HDFCBANK",
    "ICICIBANK",
    "AXISBANK",
    "KOTAKBANK",
    "INDUSINDBK",
    "BAJFINANCE",
    "PNB"
]

In [38]:
results = []

for ticker in TICKERS:
    file_path = os.path.join(DATA_FOLDER, f"{ticker}.csv")

    if not os.path.exists(file_path):
        print(f"Warning: File not found for {ticker}")
        continue

    df = pd.read_csv(file_path)

    # Ensure Date is a datetime
    df["Date"] = pd.to_datetime(df["Date"])

    # Filter for the financial year window (inclusive)
    df = df[(df["Date"] >= START_DATE) & (df["Date"] <= END_DATE)]

    if df.empty:
        print(f"Warning: No data in financial-year range for {ticker}")
        continue

    # Ensure Adj Close column exists
    if "Adj Close" not in df.columns:
        raise ValueError(f"Adj Close column missing in {ticker}.csv")

    # Compute annualized volatility
    sigma_e = compute_annualized_volatility(df)

    results.append({
        "Ticker": ticker,
        "Sigma_E": sigma_e
    })

In [39]:
vol_df = pd.DataFrame(results)

# Save results
vol_df.to_csv("volatility_results.csv", index=False)
vol_df

,Ticker,Sigma_E
0,SBIBANK,0.299478
1,BANKBARODA,0.395868
2,CANBK,0.399892
3,HDFCBANK,0.246321
4,ICICIBANK,0.286065
5,AXISBANK,0.322907
6,KOTAKBANK,0.267515
7,INDUSINDBK,0.429140
8,BAJFINANCE,0.342022
9,PNB,0.394363
